# Job Market Data Processing: Systematic Validation Interface

This notebook serves as a **systematic validation interface** for validating each step of the job market data processing pipeline. It ensures data quality, validates processing steps, and provides comprehensive diagnostics for reliable analysis.

## Purpose
- **Systematic data validation** with comprehensive quality checks
- **Step-by-step pipeline validation** of data processing stages
- **Evidence-based quality assessment** at each processing stage
- **Centralized integration** with src/ classes and Quarto export system
- **Business insight validation** for reliable reporting

## Validation Framework
This notebook implements a 5-step validation process that ensures data reliability before Quarto integration.

## Step 1: Initialize Clean Environment & Force Raw Data Loading

We'll start fresh by clearing any cached data and forcing the system to load from the original raw Lightcast CSV file.

In [1]:
# Clean environment setup for developer validation
import sys
import os
from pathlib import Path
import warnings
import importlib

warnings.filterwarnings('ignore')

# Add src to path
sys.path.append('../src')

# FORCE RELOAD MODULE TO PICK UP PATH FIXES
if 'data.spark_analyzer' in sys.modules:
    print("PROCESSING: Reloading spark_analyzer module to pick up path fixes...")
    importlib.reload(sys.modules['data.spark_analyzer'])
else:
    print("PROCESSING: Loading spark_analyzer module for the first time...")

from data.spark_analyzer import SparkJobAnalyzer, create_raw_analyzer
from data.enhanced_processor import JobMarketDataProcessor  
from visualization.simple_plots import SalaryVisualizer
    
print("SUCCESS: Enhanced classes imported successfully:")
print("   - SparkJobAnalyzer: Raw data loading & SQL analysis") 
print("   - create_raw_analyzer: Force raw data loading function")
print("   - JobMarketDataProcessor: Data cleaning & processing pipeline")
print("   - SalaryVisualizer: Visualization utilities")

# Verify the function is callable
print(f"SUCCESS: create_raw_analyzer function: {create_raw_analyzer}")

# DEBUG: Check what path the function will actually use
try:
    import inspect
    analyzer_code = inspect.getsource(create_raw_analyzer)
    print(f"\nCONFIG: Function source verification: create_raw_analyzer calls create_spark_analyzer(force_raw=True)")
    
    # Check the actual default path in the module
    import data.spark_analyzer as sa
    load_method = getattr(sa.SparkJobAnalyzer, '_load_raw_data')
    signature = inspect.signature(load_method)
    raw_path_default = signature.parameters['raw_data_path'].default
    print(f"CONFIG: Default raw data path in module: {raw_path_default}")
    
except Exception as e:
    print(f"WARNING:  Could not inspect function details: {e}")
  
clear_spark = True  # Set to False to skip clearing

if clear_spark:
    try:
        from pyspark.sql import SparkSession
        
        # Use proper public API to check for existing sessions
        active_session = SparkSession.getActiveSession()
        
        if active_session is not None:
            print("\nCLEANING: Clearing existing Spark session for clean validation...")
            active_session.stop()
            print("   SUCCESS: Active Spark session stopped")
        else:
            print("\nSUCCESS: No active Spark session found - environment is clean")
            
        # Clean up any local spark variables (defensive programming)
        # This removes spark variables that might exist in notebook memory
        spark_vars = [var for var in locals() if var.startswith('spark')]
        
        if spark_vars:
            print(f"   CLEANING: Clearing local spark variables: {spark_vars}")
            for var in spark_vars:
                del locals()[var]
            print("   SUCCESS: Local spark variables cleared")
        else:
            print("   SUCCESS: No local spark variables found")
            
    except Exception as e:
        print(f"\nWARNING:  Note: Could not clear Spark sessions: {e}")
        print("   This is usually fine - continuing with validation...")

print(f"\nTARGET: Ready for force raw data loading and validation!")

PROCESSING: Loading spark_analyzer module for the first time...
SUCCESS: Enhanced classes imported successfully:
   - SparkJobAnalyzer: Raw data loading & SQL analysis
   - create_raw_analyzer: Force raw data loading function
   - JobMarketDataProcessor: Data cleaning & processing pipeline
   - SalaryVisualizer: Visualization utilities
SUCCESS: create_raw_analyzer function: <function create_raw_analyzer at 0x74c6e4273f60>

CONFIG: Function source verification: create_raw_analyzer calls create_spark_analyzer(force_raw=True)
CONFIG: Default raw data path in module: ../../data/raw/lightcast_job_postings.csv

SUCCESS: No active Spark session found - environment is clean
   SUCCESS: No local spark variables found

TARGET: Ready for force raw data loading and validation!


In [2]:
# FORCE LOAD RAW DATA - Developer Validation Mode
print("=" * 60)
print("STARTING: FORCING RAW DATA LOAD FOR VALIDATION")
print("=" * 60)

# Define data paths to check
data_sources = {
    "raw_lightcast": "../data/raw/lightcast_job_postings.csv",
    "processed_parquet": "../data/processed/job_market_processed.parquet",
    "clean_csv": "../data/processed/clean_job_data.csv"
}

# Check what data sources exist
print("FILES: DATA SOURCE AVAILABILITY CHECK:")
print("-" * 40)
available_sources = {}
for source_name, path in data_sources.items():
    exists = Path(path).exists()
    status = "SUCCESS: EXISTS" if exists else "ERROR: MISSING"
    print(f"   {source_name:<20}: {status}")
    if exists:
        if path.endswith('.csv'):
            # For CSV files, check file size
            size_mb = Path(path).stat().st_size / (1024 * 1024)
            print(f"   {'':<20}  DATA: Size: {size_mb:.1f} MB")
        available_sources[source_name] = path

print(f"\nTARGET: DEVELOPER MODE: FORCING RAW DATA LOAD")
print("-" * 40)

STARTING: FORCING RAW DATA LOAD FOR VALIDATION
FILES: DATA SOURCE AVAILABILITY CHECK:
----------------------------------------
   raw_lightcast       : SUCCESS: EXISTS
                         DATA: Size: 683.5 MB
   processed_parquet   : ERROR: MISSING
   clean_csv           : ERROR: MISSING

TARGET: DEVELOPER MODE: FORCING RAW DATA LOAD
----------------------------------------


In [3]:
if "raw_lightcast" not in available_sources:
    print("ERROR: CRITICAL: Raw Lightcast CSV not found!")
    print("INFO: Please ensure ../data/raw/lightcast_job_postings.csv exists")
    print("STOP: Cannot proceed with validation without raw data")
else:
    # USE ENHANCED create_raw_analyzer() function
    print("PROCESSING: Using enhanced create_raw_analyzer() for FORCE RAW loading...")
    
    try:
        # This bypasses ALL processed data and forces raw CSV loading
        raw_analyzer : SparkJobAnalyzer = create_raw_analyzer()
        
        # Validate load success
        record_count = raw_analyzer.get_df().count()
        col_count = len(raw_analyzer.get_df().columns)
        
        print(f"SUCCESS: RAW DATA LOADED SUCCESSFULLY!")
        print(f"   DATA: Records: {record_count:,}")
        print(f"   LIST: Columns: {col_count}")
        print(f"   CONFIG: Method: Enhanced SparkJobAnalyzer with force_raw=True")
        
        # DISPLAY ALL COLUMNS FOR 5 ROWS - Multiple options:
        print(f"\nLIST: SAMPLE DATA (First 5 rows, ALL {col_count} columns):")
        print("-" * 60)
        
        # Option 1: Simple .show() - displays all columns by default
        raw_analyzer.get_df().show(5, truncate=False)  # truncate=False shows full content
        
        # If you want truncated display (for readability with many columns):
        # raw_analyzer.get_df().show(5, truncate=True)  # Default truncation
        
        # Option 2: Explicit column selection (if you want to be explicit)
        # all_columns = raw_analyzer.get_df().columns
        # raw_analyzer.get_df().select(*all_columns).show(5, truncate=True)

        
        # Quick data validation using enhanced validation
        print(f"\nSEARCH: ENHANCED RAW DATA VALIDATION:")
        print("-" * 35)
        
        # The enhanced analyzer already validated the data
        print("SUCCESS: Enhanced validation completed during load")
        
        # Show first few records
        print("TEXT: Sample records (first 2, key columns):")
        
        # Get a few key columns for display
        all_cols = raw_analyzer.job_data.columns
        key_cols = []
        
        # Prioritize important columns for display
        priority_cols = ['TITLE', 'COMPANY', 'LOCATION', 'SALARY_AVG_IMPUTED']
        for col in priority_cols:
            if col in all_cols:
                key_cols.append(col)
        
        # Add a few more if we have space
        if len(key_cols) < 6:
            for col in all_cols:
                if col not in key_cols and len(key_cols) < 6:
                    key_cols.append(col)
        
        if key_cols:
            raw_analyzer.job_data.select(key_cols).show(2, truncate=True)
        
        # Show schema overview
        print(f"\nCONFIG: SCHEMA OVERVIEW:")
        print(f"   Total columns: {len(all_cols)}")
        
        # Quick column type summary
        schema_summary = {}
        for field in raw_analyzer.job_data.schema.fields:
            field_type = str(field.dataType)
            schema_summary[field_type] = schema_summary.get(field_type, 0) + 1
        
        print(f"   Column types:")
        for dtype, count in schema_summary.items():
            print(f"     {dtype}: {count} columns")
        
    except Exception as e:
        print(f"ERROR: FAILED to load raw data with enhanced method: {e}")
        print("CONFIG: Debug info:")
        print(f"   Using create_raw_analyzer() function")
        print(f"   Raw file exists: {Path('../data/raw/lightcast_job_postings.csv').exists()}")
        raw_analyzer = None
        
print(f"\nSUCCESS: Force raw loading complete - ready for deep analysis!")

PROCESSING: Using enhanced create_raw_analyzer() for FORCE RAW loading...


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/27 22:57:22 WARN Utils: Your hostname, SamWin, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/27 22:57:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 22:57:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
INFO:data.spark_analyzer:SparkJobAnalyzer initialized with Spark 4.0.1
INFO:data.spark_analyzer:PROCESSING: FORCE RAW MODE: Bypassing processed data, loading from raw source
INFO:data.spark_analyzer:Loading raw Lightcast data from: ../../data/raw/lightcast_job_postings.csv
INFO:data.spark_analyzer:SUCCESS: Raw data loaded: 72,498 re

SUCCESS: RAW DATA LOADED SUCCESSFULLY!
   DATA: Records: 72,498
   LIST: Columns: 131
   CONFIG: Method: Enhanced SparkJobAnalyzer with force_raw=True

LIST: SAMPLE DATA (First 5 rows, ALL 131 columns):
------------------------------------------------------------
+----------------------------------------+-----------------+-----------------------+----------+--------+---------+--------+----------------------+---------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------------+------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Step 2: Raw Data Schema Deep Dive & Quality Assessment

Perform comprehensive analysis of the raw Lightcast data structure, identify data quality issues, and validate the schema before processing.

In [ ]:
# COMPREHENSIVE RAW DATA ANALYSIS - Developer Deep Dive
if 'raw_analyzer' in locals() and raw_analyzer is not None and raw_analyzer.job_data is not None:
    print('Raw data analysis complete!')